In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/run_ner.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/utils_ner.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/tasks.py

!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..

In [ ]:
!mkdir data
# aynı dizinde data klasöründe train,test,dev tsv dosyaları yükleniyor.
# eğer text dosyaları varsa hazır, direk run_ner.py çalıştırılabilir.

blankLineIndicator = "BlankLineIndicator"
blank = ""
firstColumnIndex = 0
secondColumnIndex = 1

In [ ]:
import pandas as pd
train_pd = pd.read_csv("./data/train.tsv",sep='\t',skip_blank_lines=False)
test_pd = pd.read_csv("./data/test.tsv",sep='\t',skip_blank_lines=False)
dev_pd = pd.read_csv("./data/dev.tsv",sep='\t',skip_blank_lines=False)

# boş satıları -X1 ile dolduruyoruz ki cümlenin ne zaman bittiğini görebilelim
train_pd.fillna(blankLineIndicator,inplace=True)
test_pd.fillna(blankLineIndicator,inplace=True)
dev_pd.fillna(blankLineIndicator,inplace=True)

In [ ]:
print(train_pd.columns)
print(test_pd.columns)
print(dev_pd.columns)

In [ ]:
train_tsv = []
for index, row in train_pd.iterrows():
            #Token Sütun İsmi
    if row[firstColumnIndex] == blankLineIndicator:
        train_tsv.append(blank)
    else:               #Token Sütun İsmi
        train_tsv.append(str(row[firstColumnIndex]) +  " " + row[secondColumnIndex])

test_tsv = []
for index, row in test_pd.iterrows():
    #Token Sütun İsmi
    if row[firstColumnIndex] == blankLineIndicator:
        test_tsv.append(blank)
    else:              #Token Sütun İsmi
        test_tsv.append(str(row[firstColumnIndex]) +  " " + row[secondColumnIndex])

dev_tsv = []
for index, row in dev_pd.iterrows():
    #Token Sütun İsmi
    if row[firstColumnIndex] == blankLineIndicator:
        dev_tsv.append(blank)
    else:           #Token Sütun İsmi
        dev_tsv.append(str(row[firstColumnIndex]) +  " " + row[secondColumnIndex])


In [ ]:
def create_txt(file_name,lines):
    file = open(file_name, 'w') 
    for line in lines:
        file.write(line + "\n") 
    file.close()

create_txt("./data/train.txt",train_tsv)
create_txt("./data/test.txt",test_tsv)
create_txt("./data/dev.txt",dev_tsv)
# txt file'a çeviriyoruz

In [ ]:
#labels.txt -> unique varlık ismi sınıflarının olduğu text dosyası

In [ ]:
import numpy as np
column_values = train_pd[["O"]].values
unique_values =  np.unique(column_values)
print(unique_values)



In [ ]:
OUTPUT_DIR = "electra-ner" 

In [ ]:
!pip install pyarrow --upgrade

!python3 run_ner.py --data_dir ./data \
--labels ./labels.txt \
--model_name_or_path fspanda/electra-medical-small-generator \
--output_dir $OUTPUT_DIR \
--max_seq_length 192 \
--num_train_epochs 3 \
--per_device_train_batch_size 16 \
--overwrite_output_dir \
--save_steps 10000 \
--seed 41 \
--do_train \
--do_eval \
--do_predict